In [4]:
import numpy as np
from scipy.io import loadmat,savemat
import os
import tensorflow as tf


In [5]:
#in and out data
dimension = 60
context_frames = 100

x = tf.placeholder(tf.float32,[None,dimension,context_frames]) #inputs
y = tf.placeholder(tf.float32,[None,2]) #labels

In [7]:
#conv layer 1
#[filter_height, filter_width, in-channels, out-channels]
weights_conv1 = tf.Variable(tf.truncated_normal(shape = [3,3,1,256], stddev = 0.01, dtype = tf.float32))
bias_conv1 = tf.Variable(tf.constant(0.1,shape = [256]))

# reshape of input -1 dynamically calculates the batch_size,in_height,in_width,in_channels
input_x = tf.reshape(x,[-1,dimension,context_frames,1])
print(input_x)
conv1 = tf.nn.conv2d(input_x,weights_conv1, strides = [1,2,2,1],padding = 'VALID')
conv1_out = tf.nn.relu(conv1+bias_conv1)
print(conv1_out)

Tensor("Reshape_1:0", shape=(?, 60, 100, 1), dtype=float32)
Tensor("Relu:0", shape=(?, 29, 49, 256), dtype=float32)


In [8]:
#conv layer 2
weights_conv2 = tf.Variable(tf.truncated_normal(shape = [3,3,256,128], stddev = 0.01, dtype = tf.float32))
bias_conv2 = tf.Variable(tf.constant(0.1,shape = [128]))

conv2 = tf.nn.conv2d(conv1_out,weights_conv2,strides = [1,2,2,1], padding = 'VALID')
conv2_out = tf.nn.relu(conv2+bias_conv2)
print(conv2_out)

Tensor("Relu_1:0", shape=(?, 14, 24, 128), dtype=float32)


In [9]:
#conv layer 3
weights_conv3 = tf.Variable(tf.truncated_normal(shape = [5,5,128,64], stddev = 0.01, dtype = tf.float32))
bias_conv3 = tf.Variable(tf.constant(0.1,shape = [64]))

conv3 = tf.nn.conv2d(conv2_out,weights_conv3,strides = [1,1,1,1], padding = 'VALID')
conv3_out = tf.nn.relu(conv3+bias_conv3)
print(conv3_out)

Tensor("Relu_2:0", shape=(?, 10, 20, 64), dtype=float32)


In [10]:
#conv layer 4
weights_conv4 = tf.Variable(tf.truncated_normal(shape = [5,5,64,64], stddev = 0.01, dtype = tf.float32))
bias_conv4 = tf.Variable(tf.constant(0.1,shape = [64]))

conv4 = tf.nn.conv2d(conv3_out,weights_conv4,strides = [1,1,1,1], padding = 'VALID')
conv4_out = tf.nn.relu(conv4+bias_conv4)
print(conv4_out)

Tensor("Relu_3:0", shape=(?, 6, 16, 64), dtype=float32)


In [11]:
#dimensionality reduce
reduced_dim = conv4_out.shape[1] * conv4_out.shape[1] * conv4_out.shape[3]
print(int(reduced_dim),type(reduced_dim))


2304 <class 'tensorflow.python.framework.tensor_shape.Dimension'>


In [12]:
#dense layer 1
weights_dense1 = tf.Variable(tf.truncated_normal([int(reduced_dim),256], stddev = 0.01, dtype = tf.float32))
bias_dense1 = tf.Variable(tf.constant(0.1,shape = [256]))

input_dense = tf.reshape(conv4_out, shape = [-1,int(reduced_dim)])
#print(input_dense)
dense1_sum = tf.matmul(input_dense,weights_dense1)+bias_dense1
dense1_output = tf.nn.relu(dense1_sum)
print(dense1_output)

Tensor("Relu_4:0", shape=(?, 256), dtype=float32)


In [13]:
#adding dropout to dense layer 1
keep_prob = tf.placeholder(dtype = tf.float32)
dense1_dropout = tf.nn.dropout(dense1_output,keep_prob)
print(dense1_dropout)

Tensor("dropout/mul:0", shape=(?, 256), dtype=float32)


In [14]:
# dense layer 2
weights_dense2 = tf.Variable(tf.truncated_normal([256,256], stddev = 0.01, dtype = tf.float32))
bias_dense2 = tf.Variable(tf.constant(0.1,shape = [256]))

dense2_sum = tf.matmul(dense1_dropout,weights_dense2)+ bias_dense2
dense2_output = tf.nn.relu(dense2_sum)
print(dense2_output)

Tensor("Relu_5:0", shape=(?, 256), dtype=float32)


In [15]:
dense2_dropout = tf.nn.dropout(dense2_output,keep_prob)
print(dense2_dropout)

Tensor("dropout_1/mul:0", shape=(?, 256), dtype=float32)


In [ ]:
#output layer
output_weights = tf.Variable(tf.truncated_normal(shape = [256,2], stddev = 0.01, dtype = tf.float32))
output_bias = tf.Variable(tf.constant(0.1,shape = [2]))

output_sum = tf.matmul(dense2_dropout,output_weights)+output_bias
final_output = tf.nn.softmax(output_sum)
print(final_output)

In [ ]:
# losses and optimizers

cross_entropy = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = final_output))
optimizer = tf.train.AdamOptimizer(1e-4)
train_step = optimizer.minimize(cross_entropy)

correct_predictions = tf.equal( tf.argmax(final_output, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean( tf.cast( correct_predictions, tf.float32 ))

In [ ]:
#initializing tensors using session

session = tf.InteractiveSession()

init = tf.global_variables_initializer()
#saver = tf.train.Saver(max_to_keep = None)

In [ ]:
def cal_eer(scores_true, scores_false):
    
    """ Calculate EER
        Args:
            scores_true: A score list of positive samples
            scores_false: A score list of negative samples
        Return:
            (EER, threshold)
        Note:
            Here, the threshold is set, and if the score is higher than or equal to the threshold,
            the decision is passed, and the decision is rejected as below.
    """
    min1 = min(scores_true)
    min2 = min(scores_false)
    low = min(min1, min2)
    max1 = max(scores_true)
    max2 = max(scores_false)
    high = max(max1, max2)
    FAR = -55
    FRR = -55
    mid = -100
    # Bisection find threshold
    while True:
        bef_FAR = FAR 
        bef_FRR = FRR
        bef_mid = mid
        mid = (high + low) / 2
        FRR = sum([1 for s in scores_true if s < mid]) / len(scores_true)
        FAR = sum([1 for s in scores_false if s >= mid]) / len(scores_false)
        #print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)

        if abs(FRR - FAR) < 1e-7:
            print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)
            break

        if FRR < FAR:
            low = mid
        else:
            high = mid
            
        if bef_FAR == FAR :
            if bef_FRR == FRR :
                if bef_mid == mid :
                    print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)
                    break

    EER  =  FRR
    return EER, (high + low) / 2

In [ ]:
from scipy.io import savemat

def get_EER(test_path, index, sp_path ):
    scores = []
    development_labels = []
    probabilities = []

    #test_path = "C:\\Users\\project1\\Desktop\\Siva\\HT_IFCC_hemlata\\CNN_input_testing_dev"
    dev_num = len(list(os.listdir(test_path)))//2
    
    for j in range(1, dev_num+1):
        filename = "features_"+str(j)+".npy"
        labelsname= "labels_"+str(j)+".npy"
        
        features = np.load(os.path.join(test_path, filename))
        labels = np.load(os.path.join(test_path, labelsname))
        patches = labels.shape[0]
        patch_prob = session.run(final_output, feed_dict = {x:features, keep_prob : 1.0})
        final_prob = np.zeros([2])
        
        for i in range(patches):
            if(patch_prob[i][0] == 0):
                patch_prob[i][0] = 1e-25
            if(patch_prob[i][1] == 0):
                patch_prob[i][1] = 1e-25
                
            final_prob[0] = final_prob[0] + np.log(patch_prob[i][0])
            final_prob[1] = final_prob[1] + np.log(patch_prob[i][1])

        final_prob = final_prob / patches
        score = (final_prob[0]) - (final_prob[1])
        audio_label = labels[0]

        scores.append(score)
        probabilities.append( np.mean(patch_prob, axis=0) )
        development_labels.append( audio_label )

    scores = np.array(scores)
    development_labels = np.array(development_labels)
    probabilities = np.array(probabilities)

    scores_geniue = []
    scores_spoof = []
    prob_genuine = []
    prob_spoof = []

    for i in range(development_labels.shape[0]):
        if development_labels[i][0] == 1 :
            scores_geniue.append(scores[i])
            prob_genuine.append(probabilities[i])
        elif development_labels[i][1] == 1:
            scores_spoof.append(scores[i])
            prob_spoof.append(probabilities[i])

    scores_geniue = np.array(scores_geniue)
    scores_spoof = np.array(scores_spoof)
    prob_genuine = np.array(prob_genuine)
    prob_spoof = np.array(prob_spoof)
    
    a,b = cal_eer(scores_geniue, scores_spoof)
    
    score_path = os.path.join(sp_path, "scores")
    prob_path = os.path.join(sp_path, "probabilities")
    
    if not os.path.exists(score_path):
        os.makedirs(score_path)
    if not os.path.exists(prob_path):
        os.makedirs(prob_path)
        
    prob_file = "probabilities_"+str(index)
    sc_file = "scores_"+str(index)
    savemat(os.path.join(score_path, sc_file), { 'genuine':scores_geniue, 'spoof':scores_spoof, 'eer' : a*100 })
    savemat(os.path.join(prob_path, prob_file), { 'genuine':prob_genuine, 'spoof':prob_spoof })
    
    return a

In [ ]:
#Training and testing accuracy
saver = tf.train.Saver(max_to_keep=None)
session.run(init)

features_path = "C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\overlapData"
development_path = "C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\devoverlapData"
evaluation_path = "C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\evaloverlapData"
model_path = "C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\Models_60D_CNN_allc2d"
    
dev_sp_path = "C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\dev_sp_60D_CNN_allc2d"
eva_sp_path = "C:\\Users\\HIMANSHU\\Desktop\\Aditya\\ASVspoof2017_CMPOC_CMVN\\eval_sp_60D_CNN_allc2d" 

if not os.path.exists(model_path):
    os.makedirs(model_path)
    
if not os.path.exists(dev_sp_path):
    os.makedirs(dev_sp_path)
    
batches = (len(list(os.listdir(features_path))))//2
    

In [ ]:
print(batches)

In [ ]:
for i in range(0,50):
    
    for j in range(1,batches+1):  #number of batches


        filename = "batch_"+str(j)+".npy"
        labelsname= "label_"+str(j)+".npy"

        features = np.load(os.path.join(features_path, filename))
        labels = np.load(os.path.join(features_path, labelsname))

        #print("before  ",str(j),"   ",accuracy.eval(feed_dict = {x:features, y:labels, keep_prob:1.0}))
        train_step.run(session = session,feed_dict={ x:features, y:labels, keep_prob:0.5})
        #print("after   ",str(j),"   ",accuracy.eval(feed_dict = {x:features, y:labels, keep_prob:1.0}))


    save_path = saver.save(session,os.path.join(model_path,"Model_"+str(i+1)))

    print("iteration  ",(i+1))
    eer_dev = get_EER(development_path, (i+1), dev_sp_path)
    print(" develop eer  ",eer_dev)
    print()
    